In [2]:
import pandas as pd
from pandas.io import sql
import sqlite3


In [3]:
# Reading CSV to Dataframe
orders = pd.read_csv('data/P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('data/P12-OrderBreakdown.csv', encoding = 'utf-8')

In [4]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [5]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


In [6]:
# Add underscores in column names
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

In [7]:
# Establishing Local DB connection
db_connection = sqlite3.connect('data/eshop.db.sqlite')

In [8]:
# save the 2 dataframes to sql tables in database
orders.to_sql('orders', con = db_connection, if_exists = 'replace', index = False)

orders_break_down.to_sql('orders_break_down', con = db_connection, if_exists = 'replace', index = False)

8047

In [9]:
# Select first 10 rows of all columns in orders table

query = """
        SELECT * 
        FROM orders 
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [10]:
# Select Number of Distinct Orders for Each Customer
query = """
        SELECT customer_name,
                COUNT(orders.order_id) AS number_of_orders
        FROM orders
        GROUP BY customer_name
        ORDER BY number_of_orders DESC
        ;
        """

orders_per_person = sql.read_sql(query, con=db_connection)
orders_per_person

,customer_name,number_of_orders
0,John Grady,13
1,Joel Jenkins,12
2,Aaron Smayling,12
3,Yoseph Carroll,11
4,Sarah Brown,11
...,...,...
787,Brooke Gillingham,1
788,Barbara Fisher,1
789,Art Ferguson,1
790,Andy Reiter,1


In [11]:
# Select Number of Customers for Each Country

query = """
        SELECT orders.country,
                COUNT(orders.customer_name) AS number_of_customers
        FROM orders
        GROUP BY orders.country
        ORDER BY number_of_customers DESC
        ;
        """

orders_per_country = sql.read_sql(query, con=db_connection)
orders_per_country

,country,number_of_customers
0,France,991
1,Germany,806
2,United Kingdom,700
3,Italy,493
4,Spain,403
5,Netherlands,194
6,Austria,135
7,Sweden,100
8,Belgium,68
9,Ireland,50


In [12]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table

query = """
        SELECT order_id,
                discount,
                sales,
                quantity
        FROM orders_break_down
        GROUP BY order_id
        ;
        """

disc_sales_quant = sql.read_sql(query, con=db_connection)
disc_sales_quant

,order_id,discount,sales,quantity
0,ES-2011-1001989,0.1,229.0,9
1,ES-2011-1010958,0.0,58.0,2
2,ES-2011-1012469,0.0,148.0,3
3,ES-2011-1043483,0.0,224.0,11
4,ES-2011-1058269,0.0,13.0,2
...,...,...,...,...
4112,IT-2014-5966070,0.5,107.0,4
4113,IT-2014-5975833,0.1,656.0,2
4114,IT-2014-5984498,0.5,83.0,1
4115,IT-2014-5989338,0.1,1603.0,9


In [13]:
# Select discount, sales, quantity, total price for Each Order Id from orders_break_down Table

query = """
        SELECT order_id,
                discount,
                sales,
                quantity,
                (sales*quantity)-discount*(sales*quantity) AS total_price
        FROM orders_break_down
        GROUP BY order_id
        ;
        """

total_price = sql.read_sql(query, con=db_connection)
total_price

,order_id,discount,sales,quantity,total_price
0,ES-2011-1001989,0.1,229.0,9,1854.9
1,ES-2011-1010958,0.0,58.0,2,116.0
2,ES-2011-1012469,0.0,148.0,3,444.0
3,ES-2011-1043483,0.0,224.0,11,2464.0
4,ES-2011-1058269,0.0,13.0,2,26.0
...,...,...,...,...,...
4112,IT-2014-5966070,0.5,107.0,4,214.0
4113,IT-2014-5975833,0.1,656.0,2,1180.8
4114,IT-2014-5984498,0.5,83.0,1,41.5
4115,IT-2014-5989338,0.1,1603.0,9,12984.3


In [14]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100

query = """
        SELECT order_id,
                (sales*quantity)-discount*(sales*quantity) AS total_price
        FROM orders_break_down
        WHERE total_price > 100
        GROUP BY order_id
        ;
        """

total_price_100 = sql.read_sql(query, con=db_connection)
total_price_100

,order_id,total_price
0,ES-2011-1001989,1854.9
1,ES-2011-1010958,116.0
2,ES-2011-1012469,444.0
3,ES-2011-1043483,2464.0
4,ES-2011-1058269,2692.8
...,...,...
3574,IT-2014-5965314,120.0
3575,IT-2014-5966070,214.0
3576,IT-2014-5975833,1180.8
3577,IT-2014-5989338,12984.3


In [15]:
# Select All Order Ids, Customers And The Products (product_name) They Have Bought

query = """
        SELECT orders.order_id,
                        orders.customer_name AS customer,
                        orders_break_down.product_name AS product
                FROM orders
                JOIN orders_break_down
                ON orders.order_id = orders_break_down.order_id
        
        ;
        """

cust_prod = sql.read_sql(query, con=db_connection)
cust_prod

,order_id,customer,product
0,IT-2011-3647632,Eugene Moren,"Enermax Note Cards, Premium"
1,ES-2011-4869686,Dorothy Dickinson,"Dania Corner Shelving, Traditional"
2,ES-2011-4939443,Arthur Prichep,"Binney & Smith Sketch Pad, Easy-Erase"
3,IT-2011-2942451,Grant Thornton,"Boston Markers, Easy-Erase"
4,IT-2011-2942451,Grant Thornton,"Eldon Folders, Single Width"
...,...,...,...
8042,ES-2014-3638865,Susan Vittorini,"Fellowes Shelving, Single Width"
8043,ES-2014-4785777,Dennis Pardue,"Wilson Jones Index Tab, Economy"
8044,IT-2014-3715679,Jim Kriz,"Avery Binder Covers, Recycled"
8045,IT-2014-3715679,Jim Kriz,"BIC Pencil Sharpener, Fluorescent"


In [16]:
# Select Number of 'Furniture' Orders For Each Country
# order, category, country

query = """
        SELECT  orders_break_down.category,
                orders.country,
                COUNT(orders_break_down.category) AS no_of_orders
            FROM orders_break_down
            JOIN orders
                ON orders_break_down.order_id = orders.order_id
            WHERE orders_break_down.category = 'Furniture'
            GROUP BY orders.country
        ;
        """

furniture = sql.read_sql(query, con=db_connection)
furniture


,category,country,no_of_orders
0,Furniture,Austria,40
1,Furniture,Belgium,20
2,Furniture,Denmark,9
3,Furniture,Finland,16
4,Furniture,France,299
5,Furniture,Germany,264
6,Furniture,Ireland,12
7,Furniture,Italy,136
8,Furniture,Netherlands,76
9,Furniture,Norway,6


In [17]:
# Select Number of 'Furniture' Orders For The Country Denmark

query = """
        SELECT  orders_break_down.category,
                orders.country,
                COUNT(orders_break_down.category) AS no_of_orders
            FROM orders_break_down
            JOIN orders
                ON orders_break_down.order_id = orders.order_id
            WHERE orders_break_down.category = 'Furniture'
            AND orders.country = 'Denmark'
            GROUP BY orders.country
        ;
        """

Denmark = sql.read_sql(query, con=db_connection)
Denmark

,category,country,no_of_orders
0,Furniture,Denmark,9


In [18]:
# Select Total Sales With Discount (discount > 0) and Without Discount (discount = 0) for Each Country

query = """
        SELECT  orders.country,
                orders_break_down.discount,
                COUNT(orders_break_down.order_id) AS no_of_orders
            FROM orders_break_down
            LEFT JOIN orders
                ON orders_break_down.order_id = orders.order_id
            WHERE orders_break_down.discount = 0
            GROUP BY orders.country
        ;
        """

No_Discount = sql.read_sql(query, con=db_connection)
No_Discount

,country,discount,no_of_orders
0,Austria,0.0,264
1,Belgium,0.0,135
2,Finland,0.0,64
3,France,0.0,1062
4,Germany,0.0,1041
5,Italy,0.0,694
6,Norway,0.0,70
7,Spain,0.0,556
8,Switzerland,0.0,78
9,United Kingdom,0.0,945


In [19]:
# Select Total Sales With Discount (discount > 0) and Without Discount (discount = 0) for Each Country

query = """
        SELECT  orders.country,
                orders_break_down.discount,
                COUNT(orders_break_down.order_id) AS no_of_orders
            FROM orders_break_down
            LEFT JOIN orders
                ON orders_break_down.order_id = orders.order_id
            WHERE orders_break_down.discount > 0
            GROUP BY orders.country
        ;
        """

Discount = sql.read_sql(query, con=db_connection)
Discount

,country,discount,no_of_orders
0,Denmark,0.5,60
1,France,0.1,854
2,Germany,0.1,599
3,Ireland,0.5,100
4,Italy,0.4,285
5,Netherlands,0.5,393
6,Portugal,0.5,70
7,Spain,0.6,205
8,Sweden,0.5,203
9,United Kingdom,0.5,369


In [20]:
# Select Total Quantity, Total Sales for Each Country

query = """
        SELECT  orders.country,
                SUM(orders_break_down.sales) AS total_sales,
                SUM(orders_break_down.quantity) AS total_quantity
            FROM orders_break_down
            LEFT JOIN orders
                ON orders_break_down.order_id = orders.order_id
            GROUP BY orders.country
        ;
        """

sales_quantity = sql.read_sql(query, con=db_connection)
sales_quantity

,country,total_sales,total_quantity
0,Austria,79382.0,973
1,Belgium,42320.0,532
2,Denmark,7763.0,204
3,Finland,20702.0,201
4,France,609683.0,7329
5,Germany,488681.0,6179
6,Ireland,15998.0,392
7,Italy,252742.0,3612
8,Netherlands,70313.0,1526
9,Norway,20529.0,261


In [21]:
# Select Top 3 Countries and Quantities Based on Total Sales
query = """
        SELECT  orders.country,
                SUM(orders_break_down.sales) AS total_sales,
                SUM(orders_break_down.quantity) AS total_quantity
            FROM orders_break_down
            LEFT JOIN orders
                ON orders_break_down.order_id = orders.order_id
            GROUP BY orders.country
            ORDER BY total_sales DESC
            LIMIT 3
        ;
        """

sales_quantity_top3 = sql.read_sql(query, con=db_connection)
sales_quantity_top3

,country,total_sales,total_quantity
0,France,609683.0,7329
1,Germany,488681.0,6179
2,United Kingdom,420497.0,4917


In [22]:
# Select Bottom 3 Countries and Sales Based On Total Quantities

query = """
        SELECT  orders.country,
                SUM(orders_break_down.sales) AS total_sales,
                SUM(orders_break_down.quantity) AS total_quantity
            FROM orders_break_down
            LEFT JOIN orders
                ON orders_break_down.order_id = orders.order_id
            GROUP BY orders.country
            ORDER BY total_sales
            LIMIT 3
        ;
        """

sales_quantity_bottom3 = sql.read_sql(query, con=db_connection)
sales_quantity_bottom3

,country,total_sales,total_quantity
0,Denmark,7763.0,204
1,Portugal,15106.0,286
2,Ireland,15998.0,392


In [23]:
# Select Average Sales By Category For The Country 'France'

query = """
        SELECT  orders.country,
                orders_break_down.category,
                AVG(orders_break_down.sales) AS average_sales
            FROM orders_break_down
            LEFT JOIN orders
                ON orders_break_down.order_id = orders.order_id
            WHERE orders.country = 'France'
            GROUP BY orders_break_down.category
            ORDER BY average_sales
        ;
        """

france = sql.read_sql(query, con=db_connection)
france

,country,category,average_sales
0,France,Office Supplies,167.217709
1,France,Furniture,582.314381
2,France,Technology,595.145078


In [24]:
# Select Country, Category and Average Sales Where Average Sales is The Highest

query = """
        SELECT  orders.country,
                orders_break_down.category,
                AVG(orders_break_down.sales) AS average_sales
            FROM orders_break_down
            LEFT JOIN orders
                ON orders_break_down.order_id = orders.order_id
            GROUP BY orders.country, orders_break_down.category
            ORDER BY average_sales DESC
            LIMIT 1
        ;
        """

highest_sales = sql.read_sql(query, con=db_connection)
highest_sales


,country,category,average_sales
0,Switzerland,Technology,902.846154
